<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/AE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle #必要的
training_data = pickle.load(open("training_data.pkl","rb")) #必要的
testing_data = pickle.load(open("testing_data.pkl","rb")) #必要的

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [2]:
 import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#先搞个label。再搞个sentence
#train_data = [training_data[i][1] for i in range(len(training_data))]

##print(train_data[0]) #hopeless for tmr :(
##test:'@AndreaMarySmith very helpful .... Or will be once I stop crying :('
'''
train_data =[]
for i in range(len(training_data)):
  train_data.append(training_data[i][1])
print(train_data[0]) 
#上下一样
'''

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos


In [4]:
#做预处理 变小写
import pprint
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]


In [5]:

import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]  #完全每句话的分词的集合。train_t[0][0]每个单词
for s in text_train:
  text_train = tknzr.tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
pprint.pprint(train_t[:5])         #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)
pprint.pprint(text_test[:5])


[['hopeless', 'for', 'tmr', ':('],
 ['everything',
  'in',
  'the',
  'kids',
  'section',
  'of',
  'ikea',
  'is',
  'so',
  'cute',
  '.',
  'shame',
  "i'm",
  'nearly',
  '19',
  'in',
  '2',
  'months',
  ':('],
 ['@hegelbon',
  'that',
  'heart',
  'sliding',
  'into',
  'the',
  'waste',
  'basket',
  '.',
  ':('],
 ['“',
  '@ketchburning',
  ':',
  'i',
  'hate',
  'japanese',
  'call',
  'him',
  '"',
  'bani',
  '"',
  ':(',
  ':(',
  '”',
  'me',
  'too'],
 ['dang', 'starting', 'next', 'week', 'i', 'have', '"', 'work', '"', ':(']]
['@danieloconnel18', 'you', 'could', 'say', 'he']


In [18]:
#theprincesszooz but i see what youre going at   
#yes  subjective pain may not be real
#  but that does not make it less painful

import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 

  return t

text_train = remove(train_t)
text_test = remove(test_t)
print(text_train[:5])
#print('==')
#pprint.pprint(test_t[:5])

[['hopeless', 'for', 'tmr', ':('], ['everything', 'in', 'the', 'kids', 'section', 'of', 'ikea', 'is', 'so', 'cute', 'shame', "i'm", 'nearly', 'in', 'months', ':('], ['hegelbon', 'that', 'heart', 'sliding', 'into', 'the', 'waste', 'basket', ':('], ['ketchburning', 'i', 'hate', 'japanese', 'call', 'him', 'bani', ':(', ':(', 'me', 'too'], ['dang', 'starting', 'next', 'week', 'i', 'have', 'work', ':(']]


In [22]:
#停用词  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_ns=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_ns.append(filtered_sentence1)
pprint.pprint(text_train_ns[:5])

text_test_ns=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_ns.append(filtered_sentence2)
pprint.pprint(text_test_ns[:5])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[['hopeless', 'tmr', ':('],
 ['everything',
  'kids',
  'section',
  'ikea',
  'cute',
  'shame',
  "i'm",
  'nearly',
  'months',
  ':('],
 ['hegelbon', 'heart', 'sliding', 'waste', 'basket', ':('],
 ['ketchburning', 'hate', 'japanese', 'call', 'bani', ':(', ':('],
 ['dang', 'starting', 'next', 'week', 'work', ':(']]
[['andreamarysmith', 'helpful', '...', 'stop', 'crying', ':('],
 ['realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'],
 ['soon',
  'tweeted',
  'planted',
  'claws',
  'thigh',
  'traction',
  'zoomed',
  'away',
  ':('],
 ['luketothestars', 'damnit', ':('],
 ['klm',
  'used',
  'pry',
  'pv',
  '...',
  'wish',
  'could',
  'relive',
  'days',
  'become',
  'nyc',
  'pv',
  'buy',
  'way',
  'communicate',
  'nyc',
  'usa',
  'klm',
  ':(']]


In [23]:
#找词根
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_train_le = []
for tokens in text_train_ns:
  lemma_sentence1 = [lemmatizer.lemmatize(w) for w in tokens ]
  text_train_le.append(lemma_sentence1)
pprint.pprint(text_train_le[:10])

text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[['hopeless', 'tmr', ':('],
 ['everything',
  'kid',
  'section',
  'ikea',
  'cute',
  'shame',
  "i'm",
  'nearly',
  'month',
  ':('],
 ['hegelbon', 'heart', 'sliding', 'waste', 'basket', ':('],
 ['ketchburning', 'hate', 'japanese', 'call', 'ban', ':(', ':('],
 ['dang', 'starting', 'next', 'week', 'work', ':('],
 ['oh', 'god', 'baby', 'face', ':(', 'httpstcofcwgvaki'],
 ['rileymcdonough', 'make', 'smile', ':('],
 ['fggstar',
  'stuartthull',
  'work',
  'neighbour',
  'motor',
  'asked',
  'said',
  'hate',
  'update',
  'search',
  ':(',
  'httptcoxvmtuikwln'],
 [':(', 'tahuodyy', 'sialan', ':(', 'httpstcohvixcrl'],
 ['athabasca',
  'glacier',
  ':-(',
  'athabasca',
  'glacier',
  'jasper',
  'jaspernationalpark',
  'alberta',
  'explorealberta',
  'httptcodzzdqmfcz']]


In [24]:
#去掉复数
from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(text_train_le)):
  for plural in text_train_le[i]:
    singles = stemmer.stem(plural)
print(text_train_le[:5])

#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(text_test_le)):
  for plural in text_test_le[i]:
    singles = stemmer.stem(plural)
print(text_test_le[:5])


[['hopeless', 'tmr', ':('], ['everything', 'kid', 'section', 'ikea', 'cute', 'shame', "i'm", 'nearly', 'month', ':('], ['hegelbon', 'heart', 'sliding', 'waste', 'basket', ':('], ['ketchburning', 'hate', 'japanese', 'call', 'ban', ':(', ':('], ['dang', 'starting', 'next', 'week', 'work', ':(']]
[['andreamarysmith', 'helpful', '...', 'stop', 'cry', ':('], ['realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'], ['soon', 'tweeted', 'planted', 'claw', 'thigh', 'traction', 'zoomed', 'away', ':('], ['luketothestars', 'damnit', ':('], ['klm', 'used', 'pry', 'pv', '...', 'wish', 'could', 'relive', 'day', 'become', 'nyc', 'pv', 'buy', 'way', 'communicate', 'nyc', 'usa', 'klm', ':(']]


#**Model**

In [ ]:
#build word embeddings model
#下载 预训练 嵌入 pre_trained embedding
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25")

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】
import numpy as np
emb_dim = word_emb_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in word_emb_model:
    emb_table.append(word_emb_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)

#WordEmbedding

In [ ]:
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  



In [25]:
# 下载Twitter data
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [26]:
#look like
twitter_samples.fileids()
print("Negative LENGTH : " +  str(len(twitter_samples.strings('negative_tweets.json'))) )
print("Positive LENGTH : " +  str(len(twitter_samples.strings('positive_tweets.json'))) )
twitter_samples.strings('positive_tweets.json')[:2]
twitter_samples.strings('negative_tweets.json')[:2]

Negative LENGTH : 5000
Positive LENGTH : 5000


['hopeless for tmr :(',
 "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :("]

In [ ]:
#putting labels
label = "neg"
neg_dataset = [(label, instance) for instance in twitter_samples.strings('negative_tweets.json')]

label = "pos"
pos_dataset = [(label, instance) for instance in twitter_samples.strings('positive_tweets.json')]

print(neg_dataset[:2])
print(pos_dataset[:2])

[('neg', 'hopeless for tmr :('), ('neg', "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(")]
[('pos', '#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'), ('pos', '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!')]


In [ ]:
#splitting dataset 分隔

training_data = neg_dataset[:4000] + pos_dataset[:4000]
testing_data = neg_dataset[4000:] + pos_dataset[4000:]


####### relatively small dataset, can be used at development phase #######
dev_training_data = neg_dataset[:100] + pos_dataset[:100]
dev_testing_data = neg_dataset[100:200] + pos_dataset[100:200]


print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data)))
print("Size of testing dataset: {0}".format(len(testing_data)))
print("------------------------------------")
print("Size of dev training dataset: {0}".format(len(dev_training_data)))
print("Size of dev testing dataset: {0}".format(len(dev_testing_data)))
print("------------------------------------")

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
Size of dev training dataset: 200
Size of dev testing dataset: 200
------------------------------------


In [ ]:
#saving dataset into pickle files
import pickle
# write to cPickle
pickle.dump(training_data, open( "training_data.pkl", "wb" ) )
pickle.dump(testing_data, open( "testing_data.pkl", "wb" ) )

####### relatively small dataset, can be used at development phase #######
pickle.dump(dev_training_data, open( "dev_training_data.pkl", "wb" ) )
pickle.dump(dev_testing_data, open( "dev_testing_data.pkl", "wb" ) )

In [ ]:
#loading dataset from pickel
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '11CfwnSn-nN66U7YtCfCiJxSh3neZmKvk'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dev_training_data.pkl')  

id = '1AjVLvsK2p25eRsWWCyIqgP7fDs78UrIv'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dev_testing_data.pkl')  

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  


import pickle

dev_training_data = pickle.load(open("dev_training_data.pkl","rb"))
dev_testing_data = pickle.load(open("dev_testing_data.pkl","rb"))

training_data = pickle.load(open("training_data.pkl","rb"))
testing_data = pickle.load(open("testing_data.pkl","rb"))

In [ ]:
'''
# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model

      [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]
'''

'''
先变成词向量。
1变成向量矩阵。
句子长度不同。要做padding
np.mean()求均值。

模型： lab3
RNN
0-1 loss（这个好）或 交叉熵
linear： 叫分类器。 输出俩是p还是n
batch-size； hiddenstates层数；维度

'''
